# Project: Image Captioning

---

In this notebook, we will train your CNN-RNN model.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, we will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values we set now will be used when training your model in **Step 2** below.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance! You are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** The NN network used in this project contains an encoder CNN and RNN with LSTM cells which was used as a decoder. Encoder was based on pretrained ResNet model with its output being fed into a linear layer to create an image feature vector of the desired size (same size as the word embedding).
The vector, along with embedded word vector is fed into a RNN layer. A simple 1-layer RNN with LSTM cells was used. Final scores were generated by passing LSTM output through another linear layer.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** Original transform was used. Resize(256) + RandomCrop(224) were used to convert the original images, which may have various resolutions and aspect ratios to a 224,224 image, which can be used in our NN. Random horizontal flip is an optional step that was used to make the network more rotation-invariant. After that, the image is converted to a pytorch tensor and normalized. Normalization values were selected using the same values that were used in all [pretrained torchvision models](https://pytorch.org/docs/stable/torchvision/models.html).

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** hidden_size was kept unchanged from its original value in notebook1. vocab_threshold was selected to be very small to get rid of nonsensical 1-word captions; this value could likely be increased to impose a stricter word limit, however there was no noticeable disadavantage in keeping this value low. Most generated captions were longer than this threshold anyway. Batch size somewhere in double digit area (16-64) seems like a good compromise between learning speed and stability. Two different embed_size value were tried: 64, 128; both performed adequatly. The final model used the value of 128. 

In general, the parameter selection can be improved by performing a thorough DOE and picking the parameter values which resulted in a best model performance. 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** Adam and SGD optimizers were used - both of them had adequate performance. Literature on this topic indicated that Adam is more likely to perform and converge faster due to the use of adaptive learning rate. So Adam optimizer was selected for our model.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

import nltk
nltk.download('punkt')

## Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 3 #2  # minimum word count threshold
vocab_from_file = True     # if True, load existing vocab file
embed_size = 128 #64       # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder

num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

#image transform
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# Specifying the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

import torch.optim as optim
# Defining the optimizer.
#optimizer = optim.SGD(params, lr=0.001, momentum=0.9) 
optimizer = optim.Adam(params, lr = 0.001)


# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.90s)
creating index...


  0%|          | 1229/414113 [00:00<01:10, 5830.82it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:00<00:00, 6804.06it/s]
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:04<00:00, 21802650.55it/s]
/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<a id='step2'></a>
## Step 2: Train your Model

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [2]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time
    
# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
            

        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

caption shape:  torch.Size([64, 15])
Epoch [1/3], Step [1/6471], Loss: 9.2145, Perplexity: 10041.4191caption shape:  torch.Size([64, 12])
Epoch [1/3], Step [2/6471], Loss: 9.1330, Perplexity: 9256.1924caption shape:  torch.Size([64, 21])
Epoch [1/3], Step [3/6471], Loss: 9.0741, Perplexity: 8726.3661caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [4/6471], Loss: 8.8459, Perplexity: 6945.8032caption shape:  torch.Size([64, 17])
Epoch [1/3], Step [5/6471], Loss: 8.7291, Perplexity: 6179.8879caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [6/6471], Loss: 8.3350, Perplexity: 4167.2136caption shape:  torch.Size([64, 10])
Epoch [1/3], Step [7/6471], Loss: 7.7513, Perplexity: 2324.5289caption shape:  torch.Size([64, 14])
Epoch [1/3], Step [8/6471], Loss: 7.0267, Perplexity: 1126.2749caption shape:  torch.Size([64, 14])
Epoch [1/3], Step [9/6471], Loss: 6.2766, Perplexity: 531.9973caption shape:  torch.Size([64, 10])
Epoch [1/3], Step [10/6471], Loss: 5.7575, Perplexity: 316.5497

Epoch [1/3], Step [83/6471], Loss: 4.5459, Perplexity: 94.2466caption shape:  torch.Size([64, 17])
Epoch [1/3], Step [84/6471], Loss: 4.6037, Perplexity: 99.8504caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [85/6471], Loss: 4.1817, Perplexity: 65.4792caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [86/6471], Loss: 4.2329, Perplexity: 68.9157caption shape:  torch.Size([64, 14])
Epoch [1/3], Step [87/6471], Loss: 4.1035, Perplexity: 60.5548caption shape:  torch.Size([64, 14])
Epoch [1/3], Step [88/6471], Loss: 4.2191, Perplexity: 67.9731caption shape:  torch.Size([64, 12])
Epoch [1/3], Step [89/6471], Loss: 4.0959, Perplexity: 60.0915caption shape:  torch.Size([64, 15])
Epoch [1/3], Step [90/6471], Loss: 4.1531, Perplexity: 63.6314caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [91/6471], Loss: 4.3506, Perplexity: 77.5268caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [92/6471], Loss: 4.0092, Perplexity: 55.1042caption shape:  torch.Size([64, 14])
Epoch [1/3

Epoch [1/3], Step [247/6471], Loss: 3.8682, Perplexity: 47.8551caption shape:  torch.Size([64, 10])
Epoch [1/3], Step [248/6471], Loss: 3.9821, Perplexity: 53.6312caption shape:  torch.Size([64, 14])
Epoch [1/3], Step [249/6471], Loss: 3.6131, Perplexity: 37.0807caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [250/6471], Loss: 3.5832, Perplexity: 35.9899caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [251/6471], Loss: 3.6645, Perplexity: 39.0377caption shape:  torch.Size([64, 12])
Epoch [1/3], Step [252/6471], Loss: 3.7214, Perplexity: 41.3226caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [253/6471], Loss: 3.6732, Perplexity: 39.3759caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [254/6471], Loss: 3.5470, Perplexity: 34.7104caption shape:  torch.Size([64, 15])
Epoch [1/3], Step [255/6471], Loss: 3.7221, Perplexity: 41.3528caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [256/6471], Loss: 3.5726, Perplexity: 35.6086caption shape:  torch.Size([64, 12])


Epoch [1/3], Step [329/6471], Loss: 3.7200, Perplexity: 41.2626caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [330/6471], Loss: 3.5720, Perplexity: 35.5883caption shape:  torch.Size([64, 14])
Epoch [1/3], Step [331/6471], Loss: 3.3647, Perplexity: 28.9250caption shape:  torch.Size([64, 14])
Epoch [1/3], Step [332/6471], Loss: 3.5761, Perplexity: 35.7339caption shape:  torch.Size([64, 15])
Epoch [1/3], Step [333/6471], Loss: 3.5414, Perplexity: 34.5146caption shape:  torch.Size([64, 16])
Epoch [1/3], Step [334/6471], Loss: 3.5454, Perplexity: 34.6521caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [335/6471], Loss: 3.3633, Perplexity: 28.8832caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [336/6471], Loss: 3.4249, Perplexity: 30.7198caption shape:  torch.Size([64, 12])
Epoch [1/3], Step [337/6471], Loss: 3.4342, Perplexity: 31.0068caption shape:  torch.Size([64, 12])
Epoch [1/3], Step [338/6471], Loss: 3.6018, Perplexity: 36.6642caption shape:  torch.Size([64, 14])


Epoch [1/3], Step [411/6471], Loss: 3.3820, Perplexity: 29.4288caption shape:  torch.Size([64, 14])
Epoch [1/3], Step [412/6471], Loss: 3.2436, Perplexity: 25.6255caption shape:  torch.Size([64, 15])
Epoch [1/3], Step [413/6471], Loss: 3.1898, Perplexity: 24.2841caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [414/6471], Loss: 3.3185, Perplexity: 27.6188caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [415/6471], Loss: 3.3428, Perplexity: 28.2977caption shape:  torch.Size([64, 14])
Epoch [1/3], Step [416/6471], Loss: 3.2327, Perplexity: 25.3474caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [417/6471], Loss: 3.3274, Perplexity: 27.8670caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [418/6471], Loss: 3.2354, Perplexity: 25.4165caption shape:  torch.Size([64, 13])
Epoch [1/3], Step [419/6471], Loss: 3.2425, Perplexity: 25.5986caption shape:  torch.Size([64, 11])
Epoch [1/3], Step [420/6471], Loss: 3.5679, Perplexity: 35.4411caption shape:  torch.Size([64, 20])


...

Epoch [3/3], Step [6130/6471], Loss: 1.8144, Perplexity: 6.1373caption shape:  torch.Size([64, 12])
Epoch [3/3], Step [6131/6471], Loss: 1.9749, Perplexity: 7.2057caption shape:  torch.Size([64, 11])
Epoch [3/3], Step [6132/6471], Loss: 1.8550, Perplexity: 6.3915caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6133/6471], Loss: 1.8855, Perplexity: 6.5894caption shape:  torch.Size([64, 13])
Epoch [3/3], Step [6134/6471], Loss: 1.7186, Perplexity: 5.5768caption shape:  torch.Size([64, 10])
Epoch [3/3], Step [6135/6471], Loss: 2.0875, Perplexity: 8.0644caption shape:  torch.Size([64, 16])
Epoch [3/3], Step [6136/6471], Loss: 2.1373, Perplexity: 8.4765caption shape:  torch.Size([64, 13])
Epoch [3/3], Step [6137/6471], Loss: 1.6897, Perplexity: 5.4180caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6138/6471], Loss: 1.9496, Perplexity: 7.0256caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6139/6471], Loss: 2.0174, Perplexity: 7.5185caption shape:  torch.Size([64, 11])


Epoch [3/3], Step [6212/6471], Loss: 2.2907, Perplexity: 9.8814caption shape:  torch.Size([64, 11])
Epoch [3/3], Step [6213/6471], Loss: 2.1070, Perplexity: 8.2235caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6214/6471], Loss: 1.8717, Perplexity: 6.4993caption shape:  torch.Size([64, 15])
Epoch [3/3], Step [6215/6471], Loss: 2.1225, Perplexity: 8.3517caption shape:  torch.Size([64, 12])
Epoch [3/3], Step [6216/6471], Loss: 1.8889, Perplexity: 6.6120caption shape:  torch.Size([64, 11])
Epoch [3/3], Step [6217/6471], Loss: 1.9167, Perplexity: 6.7988caption shape:  torch.Size([64, 12])
Epoch [3/3], Step [6218/6471], Loss: 1.7366, Perplexity: 5.6778caption shape:  torch.Size([64, 9])
Epoch [3/3], Step [6219/6471], Loss: 2.3197, Perplexity: 10.1730caption shape:  torch.Size([64, 11])
Epoch [3/3], Step [6220/6471], Loss: 1.9355, Perplexity: 6.9278caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6221/6471], Loss: 1.9038, Perplexity: 6.7112caption shape:  torch.Size([64, 13])


Epoch [3/3], Step [6294/6471], Loss: 1.9416, Perplexity: 6.9696caption shape:  torch.Size([64, 16])
Epoch [3/3], Step [6295/6471], Loss: 2.1710, Perplexity: 8.7672caption shape:  torch.Size([64, 13])
Epoch [3/3], Step [6296/6471], Loss: 1.7418, Perplexity: 5.7074caption shape:  torch.Size([64, 13])
Epoch [3/3], Step [6297/6471], Loss: 1.9416, Perplexity: 6.9698caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6298/6471], Loss: 1.9422, Perplexity: 6.9742caption shape:  torch.Size([64, 11])
Epoch [3/3], Step [6299/6471], Loss: 1.9971, Perplexity: 7.3677caption shape:  torch.Size([64, 11])
Epoch [3/3], Step [6300/6471], Loss: 1.9511, Perplexity: 7.0362
caption shape:  torch.Size([64, 16])
Epoch [3/3], Step [6301/6471], Loss: 2.1092, Perplexity: 8.2416caption shape:  torch.Size([64, 13])
Epoch [3/3], Step [6302/6471], Loss: 1.8828, Perplexity: 6.5721caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6303/6471], Loss: 1.8662, Perplexity: 6.4639caption shape:  torch.Size([64, 17])

Epoch [3/3], Step [6376/6471], Loss: 1.9298, Perplexity: 6.8879caption shape:  torch.Size([64, 16])
Epoch [3/3], Step [6377/6471], Loss: 2.0689, Perplexity: 7.9158caption shape:  torch.Size([64, 13])
Epoch [3/3], Step [6378/6471], Loss: 1.8200, Perplexity: 6.1722caption shape:  torch.Size([64, 15])
Epoch [3/3], Step [6379/6471], Loss: 1.9405, Perplexity: 6.9625caption shape:  torch.Size([64, 11])
Epoch [3/3], Step [6380/6471], Loss: 1.9330, Perplexity: 6.9105caption shape:  torch.Size([64, 13])
Epoch [3/3], Step [6381/6471], Loss: 1.8857, Perplexity: 6.5912caption shape:  torch.Size([64, 12])
Epoch [3/3], Step [6382/6471], Loss: 1.9144, Perplexity: 6.7829caption shape:  torch.Size([64, 13])
Epoch [3/3], Step [6383/6471], Loss: 2.0318, Perplexity: 7.6276caption shape:  torch.Size([64, 12])
Epoch [3/3], Step [6384/6471], Loss: 1.8964, Perplexity: 6.6621caption shape:  torch.Size([64, 12])
Epoch [3/3], Step [6385/6471], Loss: 1.8553, Perplexity: 6.3934caption shape:  torch.Size([64, 16])


Epoch [3/3], Step [6458/6471], Loss: 1.8811, Perplexity: 6.5605caption shape:  torch.Size([64, 16])
Epoch [3/3], Step [6459/6471], Loss: 2.0200, Perplexity: 7.5385caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6460/6471], Loss: 1.9455, Perplexity: 6.9972caption shape:  torch.Size([64, 12])
Epoch [3/3], Step [6461/6471], Loss: 1.8205, Perplexity: 6.1749caption shape:  torch.Size([64, 15])
Epoch [3/3], Step [6462/6471], Loss: 1.9368, Perplexity: 6.9364caption shape:  torch.Size([64, 11])
Epoch [3/3], Step [6463/6471], Loss: 1.9170, Perplexity: 6.8007caption shape:  torch.Size([64, 11])
Epoch [3/3], Step [6464/6471], Loss: 1.8536, Perplexity: 6.3828caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6465/6471], Loss: 1.8774, Perplexity: 6.5364caption shape:  torch.Size([64, 12])
Epoch [3/3], Step [6466/6471], Loss: 1.8236, Perplexity: 6.1943caption shape:  torch.Size([64, 14])
Epoch [3/3], Step [6467/6471], Loss: 1.9749, Perplexity: 7.2057caption shape:  torch.Size([64, 10])


<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.